In [ ]:
!pip install git+https://github.com/nmdl-mizo/interface_master.git@develop
!mkdir -p cif_files
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/CIS-exp.cif -o 'cif_files/CIS-exp.cif'

In [1]:
from interfacemaster.hetero_searching import hetero_searcher
from pymatgen.core.structure import Structure

In [2]:
#input substrate & film information
substrate_file, film_file = 'cif_files/Si_mp-149_conventional_standard.cif', \
'cif_files/ZnO_mp-2133_conventional_standard.cif'
substrate_structure, film_structure = list(map(lambda x:Structure.from_file(x), [substrate_file, film_file]))
substrate_name, film_name = 'Si', 'ZnO'
my_searcher = hetero_searcher(substrate_structure, film_structure, substrate_name, film_name)

/opt/anaconda3/envs/general/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


Might be time consuming with high index limitation. Recommend to start by 2. 

In [3]:
#matching
my_searcher.matching(film_max_miller = 2, substrate_max_miller = 2)

81 non-identical matchings found


In [4]:
#generating
my_searcher.generating(max_anum = 200, min_slab_length = 15)

19 interfaces generated, files saved in the dict Si_ZnO_interfaces


In [5]:
#write interface_info into json data
my_searcher.write_results('interface_info')

The searching results provides all the interfaces satsifying your requirement, you can get the interface object by calling its index.

In [10]:
selected_interface = my_searcher.interfaces[1]

Then you can customize the interface structure using this selected interface object

In [11]:
help(selected_interface.get_bicrystal)

Help on method get_bicrystal in module interfacemaster.interface_generator:

get_bicrystal(dydz=None, dx=0, dp1=0, dp2=0, xyz_1=None, xyz_2=None, vx=0, filename='POSCAR', two_D=False, filetype='VASP', mirror=False, KTI=False) method of interfacemaster.interface_generator.core instance
    generate a cif file for the bicrystal structure
    
    Parameters
    ----------
    dydz : numpy array
        translation vector in the interface
    dx : float
        translation normal to the interface
    dp1, dp2 : float
        termination of slab 1, 2
    xyz1, xyz2 : list
        expansion of slab 1, 2
    vx : float
        vacuum spacing, default 0
    filename : str
        filename, default 'POSCAR'
    two_D : bool
        whether a two CSL
    filetype : str
        filetype, 'VASP' or 'LAMMPS', default 'VASP'
    mirror : bool
        mirror, default False
    KTI : bool
        KTI, default False



In [7]:
#e.g increase interface displacing
from numpy import ceil
min_slab_length = 15
x1, x2 = int(ceil(15 / selected_interface.height_1)), int(ceil(15/selected_interface.height_2))
selected_interface.get_bicrystal(two_D = True, xyz_1 = [x1,1,1], xyz_2 = [x2,1,1], vx=5, dx=5, filename= 'POSCAR')